In [1]:
%%capture
%pip install requests pandas

In [1]:
import json
import os
from os.path import exists
import pandas as pd
import requests
from tqdm import tqdm

# Setup API for HTTPS requests

### Authentication to use TMDB API

TMDB requires users to create an account to access its API. After creating an account, you can request a bearer token, which is then used to authenticate HTTPS requests. For using this code, the provided _config.json_ file contains the neccessary credentials used in the request header. Finally, the header is parsed with each request.

In [4]:
CONFIG_PATH = SCRAPED_DATA_PATH = os.path.join(os.path.abspath(""), "config.json")
BASE_URL = "https://api.themoviedb.org/3/movie/"
BASE_URL_TOP_RATED = BASE_URL + "top_rated?"

In [5]:
# Load the config file with API credentials
if exists(CONFIG_PATH):
    with open(CONFIG_PATH) as config_file:
        config = json.load(config_file)
        APP_NAME = config["TMDB"]["TMDB_APPLICATION_NAME"]
        AUTH_USER = config["TMDB"]["TMDB_EMAIL"]
        AUTH_TOKEN = config["TMDB"]["TMDB_BEARER_KEY"]
    
    # Define the headers to include the authentication token
    HEADERS = {
        "accept": "application/json",
        "Authorization": f"Bearer {AUTH_TOKEN}",
    }
    print(f"Config found - HEADER generated from:\n{CONFIG_PATH}")

else:
    print("Config not found!")

Config found - HEADER generated from:
C:\Users\Aleksandar\School\02805_Final_project\code\config.json


In order to check if authentication is valid, the response should return 200:

In [5]:
response = requests.get(BASE_URL_TOP_RATED+"authentication", headers=HEADERS)
print(response)

<Response [200]>


# Scraping TMDB

## Scraping the top 10.000 features on TMDB

The TMDB web structure is page-based, meaning any search performed on their database returns results one page at a time. It is the user's responsibility to specify which page to request. Therefore, to retrieve all search results, it is necessary to determine the total number of pages.

### Finding the number of pages

The total number of pages can be found by using the default base url and access the _"total\_pages"_ field.

In [6]:
response = requests.get(BASE_URL_TOP_RATED, headers=HEADERS)

In [7]:
TOTAL_PAGES = response.json()["total_pages"]
print(f"{TOTAL_PAGES=}")

TOTAL_PAGES=490


### Extracting features from pages

Using the total number of pages, all features can be extracted incrementally for each page and appended to a pandas DataFrame. The DataFrame can then be saved as a .csv file. 
Initially, the DataFrame is set up as follows:

In [8]:
features_df = pd.DataFrame()

Then, using a for-loop, the incemental page-number with the parameters are parsed the HTTPS request via the TMDB API. 
The parameters used in the code return all the highest rated movies from TMDB and sort them by their average rating.
These are then stored as rows in the DataFrame.

In [9]:
# Create the tqdm progress bar
progress_bar = tqdm(range(1, TOTAL_PAGES+1), desc="Scraping TMDB")

for PAGE in progress_bar:

    params = {
        "language": "en-US",
        "page": PAGE,
        "sort_by": "vote_average.desc"
    }

    response = requests.get(
        BASE_URL_TOP_RATED, 
        headers=HEADERS, 
        params=params
    )
    
    respone_json = response.json()

    current_df = pd.json_normalize(
        respone_json, 
        record_path = "results", 
        meta = [
            "page"
        ]
    )

    features_df = pd.concat([features_df, current_df])

Scraping TMDB: 100%|███████████████████████████████████████████| 490/490 [01:09<00:00,  7.02it/s]


### Cleaning the feature data and reindex the rows

In [10]:
features_clean_df = features_df.drop_duplicates(subset = "id", keep = "first")

features_clean_df = features_clean_df.reset_index()

features_clean_df = features_clean_df.drop("index", axis=1)

In [11]:
features_clean_df = features_clean_df.assign(cast="")

features_clean_df = features_clean_df[[
    "id",
    "title",
    "original_language",
    "overview",
    "cast",
    "popularity",
    "vote_count",
    "vote_average",
    "release_date",
    "genre_ids",
    "poster_path",
    "backdrop_path",
    "adult",
    "page"
]]

features_clean_df.rename(columns={
    "id": "feature_id", 
    "popularity": "feature_popularity"
}, inplace=True)

In [12]:
features_clean_df.iloc[0]

feature_id                                                          278
title                                          The Shawshank Redemption
original_language                                                    en
overview              Imprisoned in the 1940s for the double murder ...
cast                                                                   
feature_popularity                                              180.106
vote_count                                                        27250
vote_average                                                        8.7
release_date                                                 1994-09-23
genre_ids                                                      [18, 80]
poster_path                            /9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg
backdrop_path                          /zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg
adult                                                             False
page                                                            

## Scraping the cast of features on TMDB 

Using the feature IDs, the cast can be scraped and added to the feature DataFrame. Each cast entry is stored as a list of actors.

In [13]:
cast_df = pd.DataFrame()

# Create the tqdm progress bar
progress_bar = tqdm(features_clean_df.iterrows(), total=len(features_clean_df), desc="Scraping TMDB")

for idx, row in progress_bar:

    params = {
        "language": "en-US",
    }

    movie_id = row["feature_id"]

    BASE_URL_ACTORS = BASE_URL + f"{movie_id}" + "/credits"

    response = requests.get(
        BASE_URL_ACTORS, 
        headers=HEADERS, 
        params=params
    )
    
    respone_json = response.json()

    current_df = pd.json_normalize(
        respone_json, 
        record_path = "cast",
    )
    
    current_df = current_df.assign(feature_id = movie_id)

    cast_df = pd.concat([cast_df, current_df])

Scraping TMDB: 100%|█████████████████████████████████████████| 9787/9787 [27:15<00:00,  5.99it/s]


In [14]:
cast_df = cast_df.astype({'id': int, "cast_id": int,"order": int, "gender": int, "order":int})

In [15]:
cast_df.iloc[0]

adult                                              False
gender                                                 2
id                                                   504
known_for_department                              Acting
name                                         Tim Robbins
original_name                                Tim Robbins
popularity                                        20.793
profile_path            /djLVFETFTvPyVUdrd7aLVykobof.jpg
cast_id                                                3
character                                  Andy Dufresne
credit_id                       52fe4231c3a36847f800b131
order                                                  0
feature_id                                           278
Name: 0, dtype: object

## Update feature data set with actor IDs and further cleaning

By scraping the features cast list, the starring actors can be added to the feature DataFrame as a list, indicated by their TMDB IDs.

In [16]:
# Create the tqdm progress bar
progress_bar = tqdm(cast_df.iterrows(), total=len(cast_df), desc="Adding cast to feature")

for idx, row in progress_bar:
    selected_rows = cast_df[cast_df["feature_id"] == row["feature_id"]]
    selected_rows = selected_rows.sort_values(by="id",ascending = True)
    
    cast = list(selected_rows["id"])
    
    features_clean_df.at[features_clean_df.index[features_clean_df["feature_id"] == row["feature_id"]].item(), "cast"] = cast

Adding cast to feature: 100%|██████████████████████████| 366429/366429 [02:37<00:00, 2331.13it/s]


In [17]:
features_clean_df = features_clean_df.dropna(
    subset=[
        "feature_id",
        "title",
        "overview",
        "cast",                                                   
        "feature_popularity",
        "vote_count",
        "vote_average",
        "release_date",
        "genre_ids",
        "adult",
        "page"
    ]
)

In [18]:
features_clean_df = features_clean_df.reset_index()

features_clean_df = features_clean_df.drop("index", axis=1)

### Clean the actors data and reindex the rows

In [19]:
%%capture

cast_clean_df = cast_df[[
    "id",
    "original_name",
    "popularity",
    "gender",
    "adult",
    "profile_path"
]]

cast_clean_df.rename(columns={
    "id": "actor_id", 
    "popularity": "actor_popularity",
    "profile_path": "profile_image_path"
}, inplace=True)

In [20]:
cast_clean_df = cast_clean_df.sort_values(by = "actor_id", ascending = True)

cast_clean_df = cast_clean_df.drop_duplicates(subset = "actor_id", keep = "first")

cast_clean_df = cast_clean_df.reset_index()

cast_clean_df = cast_clean_df.drop("index", axis=1)

In [21]:
cast_clean_df

,actor_id,original_name,actor_popularity,gender,adult,profile_image_path
0,1,George Lucas,16.910,2,False,/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg
1,2,Mark Hamill,22.786,2,False,/2ZulC2Ccq1yv3pemusks6Zlfy2s.jpg
2,3,Harrison Ford,42.971,2,False,/n4dwIg6NbQzeMaS1yEKKlfNJH7a.jpg
3,4,Carrie Fisher,13.609,1,False,/2vmMjXMFW9ebX9VNKPW7WD6HLOR.jpg
4,5,Peter Cushing,8.631,2,False,/if5g03wn6uvHx7F6FxXHLebKc0q.jpg
...,...,...,...,...,...,...
188373,5110023,Dan Diament,0.000,0,False,None
188374,5110027,Peter Schommer,0.000,0,False,None
188375,5110031,Ramón Mendizábal,0.000,0,False,None
188376,5110033,Mimi Pinson,0.000,0,False,None


## Using TMDB to fetch wikibase\_items

In [22]:
features_clean_df = features_clean_df.assign(wikidata_id="")

In [23]:
# Create the tqdm progress bar
progress_bar = tqdm(features_clean_df.iterrows(), total=len(features_clean_df), desc="Scraping TMDB")

for idx, row in progress_bar:
    
    movie_id = row["feature_id"]

    params = {
        "language": "en-US",
    }

    BASE_URL_ACTORS = BASE_URL + f"{movie_id}" + "/external_ids"

    response = requests.get(
        BASE_URL_ACTORS, 
        headers=HEADERS, 
        params=params
    )
    respone_json = response.json()
    
    features_clean_df.at[features_clean_df.index[features_clean_df["feature_id"] == row["feature_id"]].item(), "wikidata_id"] = respone_json["wikidata_id"]

Scraping TMDB: 100%|█████████████████████████████████████████| 9787/9787 [22:43<00:00,  7.18it/s]


In [24]:
features_clean_df = features_clean_df.dropna(
    subset=[
        "feature_id",
        "title",
        "overview",
        "cast",                                                   
        "feature_popularity",
        "vote_count",
        "vote_average",
        "release_date",
        "genre_ids",
        "adult",
        "page",
        "wikidata_id"
    ]
)

In [25]:
features_clean_df

,feature_id,title,original_language,overview,cast,feature_popularity,vote_count,vote_average,release_date,genre_ids,poster_path,backdrop_path,adult,page,wikidata_id
0,278,The Shawshank Redemption,en,Imprisoned in the 1940s for the double murder ...,"[192, 504, 2141, 2555, 4029, 5063, 6573, 6574,...",180.106,27250,8.700,1994-09-23,"[18, 80]",/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg,/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg,False,1,Q172241
1,238,The Godfather,en,"Spanning the years 1945 to 1955, a chronicle o...","[1158, 1769, 2870, 3084, 3085, 3086, 3087, 308...",184.558,20695,8.700,1972-03-14,"[18, 80]",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,False,1,Q47703
2,240,The Godfather Part II,en,In the continuing saga of the Corleone crime f...,"[380, 1004, 1158, 1769, 3085, 3087, 3092, 3093...",93.454,12483,8.572,1974-12-20,"[18, 80]",/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,False,1,Q184768
3,424,Schindler's List,en,The true story of how businessman Oskar Schind...,"[930, 1088, 2282, 2829, 3738, 3896, 3970, 4924...",80.727,15899,8.567,1993-12-15,"[18, 36, 10752]",/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,False,1,Q483941
4,389,12 Angry Men,en,The defense and the prosecution have rested an...,"[1936, 2651, 3143, 4958, 5247, 5248, 5249, 525...",59.794,8695,8.547,1957-04-10,[18],/ow3wq89wM8qd5X7hWKxiRfsFf9C.jpg,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,False,1,Q2345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9782,12142,Alone in the Dark,en,Edward Carnby is a private investigator specia...,"[1234, 2224, 5354, 10822, 10875, 27122, 32205,...",18.240,606,3.246,2005-01-28,"[28, 14, 27]",/bSxrbVCyWW077zhtpuYlo3zgyug.jpg,/lcLyZzhB1ctfdH0hGBsTFrbflqP.jpg,False,490,Q701977
9783,13805,Disaster Movie,en,"Over the course of one evening, an unsuspectin...","[19754, 20404, 28639, 34202, 47884, 54593, 589...",19.322,1024,3.200,2008-08-29,[35],/3J8XKUfhJiNzwobUZVtizXYPe8b.jpg,/5V6jAFS0Q49SI07qjyFRMYlbfR9.jpg,False,490,Q837945
9784,11059,House of the Dead,en,"Set on an island off the coast, a techno rave ...","[920, 15661, 32447, 35463, 37696, 43292, 46897...",10.801,386,3.100,2003-04-11,"[27, 28, 53]",/z2mDGbV4pLtsvSMNnmnSgoVZSWK.jpg,/aNUEHLNsNMprLZt6fjf5nqDq6er.jpg,False,490,Q705216
9785,14164,Dragonball Evolution,en,"On his 18th birthday, Goku receives a mystical...","[503, 1619, 4730, 8874, 9462, 37492, 47297, 53...",16.013,2037,2.891,2009-03-12,"[28, 12, 14, 878, 53]",/sunS9xhPnFNP5wlOWrvbpBteAB.jpg,/oHrrgAPEKpz0S1ofQntiZNrmGrM.jpg,False,490,Q653303


In [26]:
features_clean_df = features_clean_df.reset_index()

features_clean_df = features_clean_df.drop("index", axis=1)

## Saving the scraped feature data to a .csv file

In [27]:
SCRAPED_FEATURES_PATH = os.path.join(os.path.abspath(""), "TMDB_scraped_features.csv")

In [28]:
SCRAPED_FEATURES_PATH

'/home/aleksandar-lukic/school/02805_Final_project/code/TMDB_scraped_features.csv'

In [29]:
features_clean_df.to_csv(SCRAPED_FEATURES_PATH, index = False)

## Saving the scraped actors data to a .csv file

In [30]:
SCRAPED_ACTORS_PATH = os.path.join(os.path.abspath(""), "TMDB_scraped_actors.csv")

In [31]:
SCRAPED_ACTORS_PATH

'/home/aleksandar-lukic/school/02805_Final_project/code/TMDB_scraped_actors.csv'

In [32]:
cast_clean_df.to_csv(SCRAPED_ACTORS_PATH, index = False)

# Get genre names by id from TMDB

Since the genres are represented in the API responses by their TMDB IDs, their english names can be requested. To ensure that all possible genres are accounted for, the TV genres are also requested. The genres are then lastely converted to DataFrames, concatenated and finally save to a csv-file.

In [14]:
# Define the URLs for the requests
MOVIE_GENRE_LIST_URL = "https://api.themoviedb.org/3/genre/movie/list?"
TV_GENRE_LIST_URL = "https://api.themoviedb.org/3/genre/tv/list?"

# Return the English genre names
params = {
    "language": "en"
}

# Send HTTPS request through the TMDB API
response_movies = requests.get(MOVIE_GENRE_LIST_URL, headers=HEADERS, params=params)
response_tv = requests.get(TV_GENRE_LIST_URL, headers=HEADERS, params=params)

# Create DataFrames and concatenate them into a single one
genre_movie_df = pd.DataFrame(response_movies.json()["genres"])
genre_tv_df = pd.DataFrame(response_tv.json()["genres"])
genres_df = pd.concat([genre_movie_df, genre_tv_df])

# Sort, Clean and Reindex the genres
genres_clean_df = genres_df.sort_values(by = "id", ascending = True)
genres_clean_df = genres_clean_df.drop_duplicates(subset = "id", keep = "first")
genres_clean_df = genres_clean_df.reset_index()
genres_clean_df = genres_clean_df.drop("index", axis=1)

genres_clean_df

,id,name
0,12,Adventure
1,14,Fantasy
2,16,Animation
3,18,Drama
4,27,Horror
5,28,Action
6,35,Comedy
7,36,History
8,37,Western
9,53,Thriller


In [15]:
TMDB_GENRES_PATH = os.path.join(os.path.abspath(""), "TMDB_genres.csv")
genres_clean_df.to_csv(TMDB_GENRES_PATH, index = False)